In [1]:
!git clone https://github.com/SalehiSina/Steamboat.git

Cloning into 'Steamboat'...
remote: Enumerating objects: 463, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 463 (delta 70), reused 74 (delta 36), pack-reused 332 (from 1)
Receiving objects: 100% (463/463), 9.17 MiB | 18.59 MiB/s, done.
Resolving deltas: 100% (250/250), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install squidpy scanpy

# Train on mouse brain data

This could take several hours. The trained model is provided, which can be used to try the interpretation part without training again.

In [4]:
import torch

# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [5]:
import os
import sys
sys.path.append("/content/Steamboat")
if torch.cuda.is_available():
    device = "cuda"
    print(torch.cuda.get_device_name(0))
else:
    device = "cpu"
    print("Using CPU")


Tesla T4


In [6]:
import scanpy as sc
import squidpy as sq
import pandas as pd
from tqdm.notebook import tqdm
import scipy as sp
import numpy as np
import multiprocessing
import pickle as pkl
import torch
import gc
import sklearn.metrics

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['font.family'] = 'arial'

pltkw = dict(bbox_inches='tight', transparent=True)

/usr/lib/python3.12/importlib/__init__.py:90: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/usr/local/lib/python3.12/dist-packages/anndata/__init__.py:44: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  return module_get_attr_redirect(attr_name, deprecated_mapping=_DEPRECATED)


In [7]:
import steamboat as sf
import steamboat.model as sm
#importlib.reload(sm)
#import steamboat.integrated_model
# importlib.reload(spaceformer.benchmarks)

In [8]:
import importlib
import steamboat.tools
#importlib.reload(steamboat.tools)

In [9]:
gc.collect()

40209

In [15]:
##Working with google colab
if False:
    adata = sc.read_h5ad("/content/drive/MyDrive/Thesis/Data/Steamboat_Data/Ex2_mouse_brain/Zhuang-ABCA-1-raw.h5ad")
    obs = pd.read_csv("/content/drive/MyDrive/Thesis/Data/Steamboat_Data/Ex2_mouse_brain/label.csv/label.csv", index_col=0)

    # obs = obs[obs['brain_section_label'].isin(obs['brain_section_label'].value_counts()[:10].index.tolist())]
    # adata = adata[obs.index, :]
    # adata.obs = obs
    # gc.collect()

    adata = adata[obs.index, :]
    gc.collect()
    adata.obs = obs
    gc.collect()

    adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()

    adata.write_h5ad("/content/drive/MyDrive/Thesis/Data/Steamboat_Data/Ex2_mouse_brain/Zhuang-ABCA-1-labeled.h5ad")
elif True:

    # Open in backed mode (does not load all data)
    adata_backed = sc.read_h5ad("/content/drive/MyDrive/Thesis/Data/Steamboat_Data/Ex2_mouse_brain/Zhuang-ABCA-1-labeled.h5ad", backed="r")

    # Load only the first n cells
    n = 100000
    adata = adata_backed[:n, :].to_memory()
else:
    adata = sc.read_h5ad("/content/drive/MyDrive/Thesis/Data/Steamboat_Data/Ex2_mouse_brain/Zhuang-ABCA-1-labeled.h5ad")


In [ ]:
##Working with laptop
if False:
    adata = sc.read_h5ad("C:\\Users\\User\Desktop\Thesis\Data\Steamboat_Data\Ex2_mouse_brain\Zhuang-ABCA-1-raw.h5ad")
    obs = pd.read_csv("C:\\Users\\User\Desktop\Thesis\Data\Steamboat_Data\Ex2_mouse_brain\label\label.csv", index_col=0)

    # obs = obs[obs['brain_section_label'].isin(obs['brain_section_label'].value_counts()[:10].index.tolist())]
    # adata = adata[obs.index, :]
    # adata.obs = obs
    # gc.collect()

    adata = adata[obs.index, :]
    gc.collect()
    adata.obs = obs
    gc.collect()

    adata.obsm['spatial'] = adata.obs[['x', 'y']].to_numpy()

    adata.write_h5ad("C:\\Users\\User\Desktop\Thesis\Data\Steamboat_Data\Ex2_mouse_brain\Zhuang-ABCA-1-labeled.h5ad")

elif True:

    # Open in backed mode (does not load all data)
    adata_backed = sc.read_h5ad("C:\\Users\\User\Desktop\Thesis\Data\Steamboat_Data\Ex2_mouse_brain\Zhuang-ABCA-1-labeled.h5ad", backed="r")

    # Load only the first n cells
    n = 1000
    adata = adata_backed[:n, :].to_memory()

else:
    adata = sc.read_h5ad("C:\\Users\\User\Desktop\Thesis\Data\Steamboat_Data\Ex2_mouse_brain\Zhuang-ABCA-1-labeled.h5ad")


In [16]:
adata

AnnData object with n_obs × n_vars = 100000 × 1122
    obs: 'brain_section_label', 'feature_matrix_label', 'donor_label', 'donor_genotype', 'donor_sex', 'cluster_alias', 'x', 'y', 'z', 'subclass_confidence_score', 'cluster_confidence_score', 'high_quality_transfer', 'neurotransmitter', 'class', 'subclass', 'supertype', 'cluster', 'neurotransmitter_color', 'class_color', 'subclass_color', 'supertype_color', 'cluster_color', 'x_ccf', 'y_ccf', 'z_ccf', 'parcellation_index', 'parcellation_organ', 'parcellation_category', 'parcellation_division', 'parcellation_structure', 'parcellation_substructure', 'parcellation_organ_color', 'parcellation_category_color', 'parcellation_division_color', 'parcellation_structure_color', 'parcellation_substructure_color'
    var: 'gene_symbol'
    obsm: 'spatial'

In [ ]:
####SINA
# to see what's inside:

adata.obs.head()
#adata.obs.columns
#adata.var["gene_symbol"].unique().shape
#adata.var["gene_symbol"].to_list()[:20]

,brain_section_label,feature_matrix_label,donor_label,donor_genotype,donor_sex,cluster_alias,x,y,z,subclass_confidence_score,...,parcellation_organ,parcellation_category,parcellation_division,parcellation_structure,parcellation_substructure,parcellation_organ_color,parcellation_category_color,parcellation_division_color,parcellation_structure_color,parcellation_substructure_color
cell_label,,,,,,,,,,,,,,,,,,,,,
182941331246012878296807398333956011710,Zhuang-ABCA-1.089,Zhuang-ABCA-1,Zhuang-ABCA-1,wt/wt,F,704,0.682522,3.366483,7.82953,0.969933,...,unassigned,unassigned,unassigned,unassigned,unassigned,#000000,#000000,#000000,#000000,#000000
221260934538535633595532020856387724686,Zhuang-ABCA-1.089,Zhuang-ABCA-1,Zhuang-ABCA-1,wt/wt,F,5243,0.667690,3.442241,7.82953,0.850554,...,unassigned,unassigned,unassigned,unassigned,unassigned,#000000,#000000,#000000,#000000,#000000
22228792606814781533240955623030943708,Zhuang-ABCA-1.089,Zhuang-ABCA-1,Zhuang-ABCA-1,wt/wt,F,14939,0.638731,3.474328,7.82953,0.888285,...,unassigned,unassigned,unassigned,unassigned,unassigned,#000000,#000000,#000000,#000000,#000000
272043042552227961220474294517855477150,Zhuang-ABCA-1.089,Zhuang-ABCA-1,Zhuang-ABCA-1,wt/wt,F,14939,0.653425,3.433218,7.82953,0.900000,...,unassigned,unassigned,unassigned,unassigned,unassigned,#000000,#000000,#000000,#000000,#000000
110116287883089187971185374239350249328,Zhuang-ABCA-1.089,Zhuang-ABCA-1,Zhuang-ABCA-1,wt/wt,F,5254,0.623896,3.513574,7.82953,0.999978,...,unassigned,unassigned,unassigned,unassigned,unassigned,#000000,#000000,#000000,#000000,#000000


In [ ]:
####SINA
adata.var_names.to_list()[-5:]

['ENSMUSG00000015619',
 'ENSMUSG00000070047',
 'ENSMUSG00000002266',
 'ENSMUSG00000036111',
 'ENSMUSG00000033063']

In [ ]:
####SINA

adata.X[20:30, -5:]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
####SINA

adata.shape      # (cells, genes)
#adata.n_vars

(2616328, 1122)

In [ ]:
####SINA

adata.obsm.keys()

KeysView(AxisArrays with keys: spatial)

In [17]:
####SINA

adata.obs['brain_section_label'].unique()

['Zhuang-ABCA-1.089', 'Zhuang-ABCA-1.077', 'Zhuang-ABCA-1.087', 'Zhuang-ABCA-1.072']
Categories (4, object): ['Zhuang-ABCA-1.072', 'Zhuang-ABCA-1.077', 'Zhuang-ABCA-1.087',
                         'Zhuang-ABCA-1.089']

In [18]:
if False:
    n_genes = 100
    np.random.seed(0)
    chosen_gene_mask = np.zeros(adata.shape[1])
    chosen_gene_mask[:n_genes] = 1.
    np.random.shuffle(chosen_gene_mask)
    chosen_gene_mask = chosen_gene_mask > 0.
    adata = adata[:, chosen_gene_mask]
gc.collect()

# Normalizing to median total counts
sc.pp.normalize_total(adata)
# Logarithmize the data
sc.pp.log1p(adata)

In [19]:
adatas = []
for i in adata.obs['brain_section_label'].unique():
    adatas.append(adata[adata.obs['brain_section_label'] == i])
    adatas[-1].obs['global'] = 0

/tmp/ipython-input-860651538.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adatas[-1].obs['global'] = 0
/tmp/ipython-input-860651538.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adatas[-1].obs['global'] = 0
/tmp/ipython-input-860651538.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adatas[-1].obs['global'] = 0
/tmp/ipython-input-860651538.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adatas[-1].obs['global'] = 0


In [20]:
adatas = sf.prep_adatas(adatas, norm=False, log1p=False)

  0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
dataset = sf.make_dataset(adatas, sparse_graph=True, regional_obs=['global'])

Using ['global'] as regional annotations.


  0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
sf.set_random_seed(0)
model = sm.Steamboat(adata.var_names.tolist(), n_heads=50, n_scales=3)
model = model.to(device)
# model.load_state_dict(torch.load('saved_models/mmbrain_new.pth', weights_only=True))


In [ ]:
model.fit(dataset, entry_masking_rate=0.1, feature_masking_rate=0.1,
          max_epoch=10000,
          loss_fun=torch.nn.MSELoss(reduction='sum'),
          opt=torch.optim.Adam, opt_args=dict(lr=0.1), stop_eps=1e-3, report_per=200, stop_tol=200, device=device)

In [25]:
"""
Input and Output of the model:

Input:

Graph structure (adj_list, regional_adj_lists)

Cell features (x, masked_x)

Regional features (regional_xs)

Output:

Reconstructed cell features ([n_cells, n_genes])

attention weights + embeddings (dict)

"""

model

Steamboat(
  (spatial_gather): BilinearAttention(
    (bias): NonNegBias(
      (elu): ELU(alpha=1.0)
    )
    (q): NonNegLinear(
      (elu): ELU(alpha=1.0)
    )
    (k_local): NonNegLinear(
      (elu): ELU(alpha=1.0)
    )
    (k_regionals): ModuleList(
      (0): NonNegLinear(
        (elu): ELU(alpha=1.0)
      )
    )
    (w_ego): NonNegScale(
      (elu): ELU(alpha=1.0)
    )
    (w_local): NonNegScale3(
      (elu): ELU(alpha=1.0)
    )
    (w_global): NonNegScale3(
      (elu): ELU(alpha=1.0)
    )
    (tanh): Tanh()
    (v): NonNegLinear(
      (elu): ELU(alpha=1.0)
    )
    (cosine_similarity): CosineSimilarity()
  )
)

In [26]:
##working with google colab
if False:
    torch.save(model.state_dict(), '/content/Steamboat/examples/saved_models/mmbrain_new.pth')
else:
    model.load_state_dict(torch.load('/content/Steamboat/examples/saved_models/mmbrain.pth',weights_only=True))

In [ ]:
##working with laptop
if False:
    torch.save(model.state_dict(), 'saved_models/mmbrain_new.pth')
else:
    model.load_state_dict(torch.load('saved_models/mmbrain.pth', map_location=torch.device('cpu'),weights_only=True))

In [27]:
# Ego, local, or global?

def annotate_adatas(adatas: list[sc.AnnData], dataset, model,
                    device=device, get_recon=False):
    """_summary_

    :param adatas: _description_
    :param dataset: _description_
    :param model: _description_
    :param device: _description_, defaults to 'cuda'
    """
    # Safeguards
    assert len(adatas) == len(dataset), "mismatch in lenghths of adatas and dataset"
    for adata, data in zip(adatas, dataset):
        assert adata.shape[0] == data[0].shape[0], f"adata[{i}] has {adata.shape[0]} cells but dataset[{i}] has {data[0].shape[0]}."

    for i, (x, adj_list, regional_xs, regional_adj_lists) in tqdm(enumerate(dataset), total=len(dataset)):
        adj_list = adj_list.squeeze(0).to(device)
        x = x.squeeze(0).to(device)
        regional_adj_lists = [regional_adj_list.to(device) for regional_adj_list in regional_adj_lists]
        regional_xs = [regional_x.to(device) for regional_x in regional_xs]

        with torch.no_grad():
            res, details = model(adj_list, x, x, regional_adj_lists, regional_xs, get_details=True)

            if get_recon:
                adatas[i].obsm['X_recon'] = res.cpu().numpy()

            adatas[i].obsm['q_score'] = details['embq'].cpu().numpy()

            adatas[i].obsm['k_score'] = details['embk'][0].cpu().numpy()
            adatas[i].obsm['k_score_global'] = model.spatial_gather.k_regionals[0](x).cpu().numpy()
            for j, emb in enumerate(details['embk'][1]):
                adatas[i].uns[f'embk_{j}'] = emb.cpu().numpy()
            adatas[i].obsm['attn'] = details['attn'].cpu().numpy()
            adatas[i].obsm['ego_attn'] = details['attnm'][0].cpu().numpy()
            adatas[i].obsm['local_attn'] = details['attnm'][1].cpu().numpy()
            for j, matrix in enumerate(details['attnm'][2]):
                adatas[i].obsm[f'regional_attn_{j}'] = matrix.cpu().numpy()

            # local attention (as graph)
            for j in range(model.spatial_gather.n_heads):
                w = details['attnp'][1].cpu().numpy()[:, j, :].flatten()
                # print(details['attnp'][1].cpu().shape)
                # print(len(w))
                uv = adj_list.cpu().numpy()
                u = uv[0]
                v = uv[1]
                # print(len(u))
                # print(len(v))
                if uv.shape[0] == 3: # masked for unequal neighbors
                    m = (uv[2] > 0)
                    w, u, v = w[m], u[m], v[m]
                adatas[i].obsp[f'local_attn_{j}'] = sp.sparse.csr_matrix((w, (u, v)),
                                                                            shape=(adatas[i].shape[0],
                                                                                adatas[i].shape[0]))

In [28]:
annotate_adatas(adatas, dataset, model, get_recon=True)

  0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
v_weights = model.spatial_gather.v.weight.detach().cpu().numpy().sum(axis=0)
adata.uns['steamboat_v_weights'] = v_weights

In [30]:
## For all samples:
def gather_obs(adata, adatas):
    all_embq = []
    all_embk = []
    all_embk_glb = []
    all_ego_attn = []
    all_local_attn = []
    all_global_attn = []
    all_attn = []
    all_recon = []
    for i in range(len(adatas)):
        all_embq.append(adatas[i].obsm['q_score'])
        all_embk.append(adatas[i].obsm['k_score'])
        all_ego_attn.append(adatas[i].obsm['ego_attn'])
        all_local_attn.append(adatas[i].obsm['local_attn'])
        all_global_attn.append(adatas[i].obsm['regional_attn_0'])
        all_recon.append(adatas[i].obsm['X_recon'])
        all_attn.append(adatas[i].obsm['attn'])
        all_embk_glb.append(adatas[i].obsm['k_score_global'])
    adata.obsm['q_score'] = np.vstack(all_embq)
    adata.obsm['k_score'] = np.vstack(all_embk)
    adata.obsm['ego_attn'] = np.vstack(all_ego_attn)
    adata.obsm['local_attn'] = np.vstack(all_local_attn)
    adata.obsm['global_attn'] = np.vstack(all_global_attn)
    adata.obsm['X_recon'] = np.vstack(all_recon)
    adata.obsm['attn'] = np.vstack(all_attn)
    adata.obsm['k_score_global'] = np.vstack(all_embk_glb)
gather_obs(adata, adatas)

In [ ]:
adata.write_h5ad("saved_h5ad/mmbrain.h5ad")

In [ ]:
for i, adata in enumerate(tqdm(adatas)):
    adata.write_h5ad(f"saved_h5ad/mmbrain_{i}.h5ad")

  0%|          | 0/129 [00:00<?, ?it/s]